In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 380)

import math, os
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
# from xgboost import XGBRegressor

# data visualization
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import pyplot as plt, style
import seaborn as sns
from seaborn import heatmap

# Algorithms
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

In [2]:
data = pd.read_csv('data.csv')
for col_name in data.columns:
        if str(col_name[:7]) == 'Unnamed':
            del data[col_name]
            
print (data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30457 entries, 0 to 30456
Columns: 373 entries, CEN_2011 to electric_category
dtypes: float64(373)
memory usage: 86.7 MB
None


,CEN_2011,eH,HH,01VIIRS_10th,02VIIRS_10th,03VIIRS_10th,04VIIRS_10th,05VIIRS_10th,06VIIRS_10th,07VIIRS_10th,08VIIRS_10th,09VIIRS_10th,10VIIRS_10th,11VIIRS_10th,12VIIRS_10th,01VIIRS_25th,02VIIRS_25th,03VIIRS_25th,04VIIRS_25th,05VIIRS_25th,06VIIRS_25th,07VIIRS_25th,08VIIRS_25th,09VIIRS_25th,10VIIRS_25th,11VIIRS_25th,12VIIRS_25th,01VIIRS_75th,02VIIRS_75th,03VIIRS_75th,04VIIRS_75th,05VIIRS_75th,06VIIRS_75th,07VIIRS_75th,08VIIRS_75th,09VIIRS_75th,10VIIRS_75th,11VIIRS_75th,12VIIRS_75th,01VIIRS_90th,02VIIRS_90th,03VIIRS_90th,04VIIRS_90th,05VIIRS_90th,06VIIRS_90th,07VIIRS_90th,08VIIRS_90th,09VIIRS_90th,10VIIRS_90th,11VIIRS_90th,12VIIRS_90th,01VIIRS_MAX,02VIIRS_MAX,03VIIRS_MAX,04VIIRS_MAX,05VIIRS_MAX,06VIIRS_MAX,07VIIRS_MAX,08VIIRS_MAX,09VIIRS_MAX,10VIIRS_MAX,11VIIRS_MAX,12VIIRS_MAX,01VIIRS_MED,02VIIRS_MED,03VIIRS_MED,04VIIRS_MED,05VIIRS_MED,06VIIRS_MED,07VIIRS_MED,08VIIRS_MED,09VIIRS_MED,10VIIRS_MED,11VIIRS_MED,12VIIRS_MED,01VIIRS_MIN,02VIIRS_MIN,03VIIRS_MIN,04VIIRS_MIN,05VIIRS_MIN,06VIIRS_MIN,07VIIRS_MIN,08VIIRS_MIN,09VIIRS_MIN,10VIIRS_MIN,11VIIRS_MIN,12VIIRS_MIN,01VIIRS_STD_DEV,02VIIRS_STD_DEV,03VIIRS_STD_DEV,04VIIRS_STD_DEV,05VIIRS_STD_DEV,06VIIRS_STD_DEV,07VIIRS_STD_DEV,08VIIRS_STD_DEV,09VIIRS_STD_DEV,10VIIRS_STD_DEV,11VIIRS_STD_DEV,12VIIRS_STD_DEV,01EVI_10th,01NDVI_10th,02EVI_10th,02NDVI_10th,03EVI_10th,03NDVI_10th,04EVI_10th,04NDVI_10th,05EVI_10th,05NDVI_10th,06EVI_10th,06NDVI_10th,07EVI_10th,07NDVI_10th,08EVI_10th,08NDVI_10th,09EVI_10th,09NDVI_10th,10EVI_10th,10NDVI_10th,11EVI_10th,11NDVI_10th,12EVI_10th,12NDVI_10th,01EVI_25th,01NDVI_25th,02EVI_25th,02NDVI_25th,03EVI_25th,03NDVI_25th,04EVI_25th,04NDVI_25th,05EVI_25th,05NDVI_25th,06EVI_25th,06NDVI_25th,07EVI_25th,07NDVI_25th,08EVI_25th,08NDVI_25th,09EVI_25th,09NDVI_25th,10EVI_25th,10NDVI_25th,11EVI_25th,11NDVI_25th,12EVI_25th,12NDVI_25th,01EVI_75th,01NDVI_75th,02EVI_75th,02NDVI_75th,03EVI_75th,03NDVI_75th,04EVI_75th,04NDVI_75th,05EVI_75th,05NDVI_75th,06EVI_75th,06NDVI_75th,07EVI_75th,07NDVI_75th,08EVI_75th,08NDVI_75th,09EVI_75th,09NDVI_75th,10EVI_75th,10NDVI_75th,11EVI_75th,11NDVI_75th,12EVI_75th,12NDVI_75th,01EVI_90th,01NDVI_90th,02EVI_90th,02NDVI_90th,03EVI_90th,03NDVI_90th,04EVI_90th,04NDVI_90th,05EVI_90th,05NDVI_90th,06EVI_90th,06NDVI_90th,07EVI_90th,07NDVI_90th,08EVI_90th,08NDVI_90th,09EVI_90th,09NDVI_90th,10EVI_90th,10NDVI_90th,11EVI_90th,11NDVI_90th,12EVI_90th,12NDVI_90th,01EVI_MAX,01NDVI_MAX,02EVI_MAX,02NDVI_MAX,03EVI_MAX,03NDVI_MAX,04EVI_MAX,04NDVI_MAX,05EVI_MAX,05NDVI_MAX,06EVI_MAX,06NDVI_MAX,07EVI_MAX,07NDVI_MAX,08EVI_MAX,08NDVI_MAX,09EVI_MAX,09NDVI_MAX,10EVI_MAX,10NDVI_MAX,11EVI_MAX,11NDVI_MAX,12EVI_MAX,12NDVI_MAX,01EVI_MED,01NDVI_MED,02EVI_MED,02NDVI_MED,03EVI_MED,03NDVI_MED,04EVI_MED,04NDVI_MED,05EVI_MED,05NDVI_MED,06EVI_MED,06NDVI_MED,07EVI_MED,07NDVI_MED,08EVI_MED,08NDVI_MED,09EVI_MED,09NDVI_MED,10EVI_MED,10NDVI_MED,11EVI_MED,11NDVI_MED,12EVI_MED,12NDVI_MED,01EVI_MIN,01NDVI_MIN,02EVI_MIN,02NDVI_MIN,03EVI_MIN,03NDVI_MIN,04EVI_MIN,04NDVI_MIN,05EVI_MIN,05NDVI_MIN,06EVI_MIN,06NDVI_MIN,07EVI_MIN,07NDVI_MIN,08EVI_MIN,08NDVI_MIN,09EVI_MIN,09NDVI_MIN,10EVI_MIN,10NDVI_MIN,11EVI_MIN,11NDVI_MIN,12EVI_MIN,12NDVI_MIN,01EVI_STD_DEV,01NDVI_STD_DEV,02EVI_STD_DEV,02NDVI_STD_DEV,03EVI_STD_DEV,03NDVI_STD_DEV,04EVI_STD_DEV,04NDVI_STD_DEV,05EVI_STD_DEV,05NDVI_STD_DEV,06EVI_STD_DEV,06NDVI_STD_DEV,07EVI_STD_DEV,07NDVI_STD_DEV,08EVI_STD_DEV,08NDVI_STD_DEV,09EVI_STD_DEV,09NDVI_STD_DEV,10EVI_STD_DEV,10NDVI_STD_DEV,11EVI_STD_DEV,11NDVI_STD_DEV,12EVI_STD_DEV,12NDVI_STD_DEV,POP_DENS_10th,POP_DENS_25th,POP_DENS_75th,POP_DENS_90th,POP_DENS_MIN,POP_DENS_MAX,POP_DENS_MED,POP_DENS_STD_DEV,B1_10th,B10_10th,B11_10th,B2_10th,B3_10th,B4_10th,B5_10th,B6_10th,B7_10th,B1_25th,B10_25th,B11_25th,B2_25th,B3_25th,B4_25th,B5_25th,B6_25th,B7_25th,B1_75th,B10_75th,B11_75th,B2_75th,B3_75th,B4_75th,B5_75th,B6_75th,B7_75th,B1_90th,B10_90th,B11_90th,B2_90th,B3_90th,B4_90th,B5_90th,B6_90th,B7_90th,B1_MAX,B10_MAX,B11_MAX,B2_MAX,B3_MAX,B4_MAX,B5_MAX,B6_MAX,B7_MAX,B1_MED,B10_MED,B11_MED,B2_MED,B3_MED,B4_MED,B5_MED,B6_M

In [3]:
df0 = data[data.electric_category == 0].drop(['CEN_2011','eH','HH', 'electric_category', 'percentage_electrified'], axis = 1)
df1 = data[data.electric_category == 1].drop(['CEN_2011','eH','HH', 'electric_category', 'percentage_electrified'], axis = 1)

In [4]:
meds = []
for col in df0: 
    meds.append(np.median(df0[col]))

len(meds)

368

In [5]:
meds2 = []
for col in df1: 
    meds2.append(np.median(df1[col]))
    
len(meds2)

368

In [6]:
d = {'feature': [col for col in df0], '0medians': meds, '1medians': meds2}
df = pd.DataFrame(data=d)

In [7]:
df['diff'] = df['1medians'] - df['0medians']

In [8]:
sorted_df = df.sort_values('diff', ascending = False)

In [9]:
sorted_df.head(60)

,0medians,1medians,feature,diff
293,996.000000,1323.000000,POP_DENS_MAX,327.000000
291,538.000000,828.500000,POP_DENS_90th,290.500000
290,335.000000,538.000000,POP_DENS_75th,203.000000
294,242.000000,355.000000,POP_DENS_MED,113.000000
289,168.000000,262.000000,POP_DENS_25th,94.000000
288,148.000000,195.000000,POP_DENS_10th,47.000000
292,130.000000,156.000000,POP_DENS_MIN,26.000000
295,65.506217,83.449086,POP_DENS_STD_DEV,17.942869
58,1.093370,1.704388,11VIIRS_MAX,0.611018
46,0.895157,1.442993,11VIIRS_90th,0.547836


In [10]:
list(sorted_df['feature'][:15])

['POP_DENS_MAX',
 'POP_DENS_90th',
 'POP_DENS_75th',
 'POP_DENS_MED',
 'POP_DENS_25th',
 'POP_DENS_10th',
 'POP_DENS_MIN',
 'POP_DENS_STD_DEV',
 '11VIIRS_MAX',
 '11VIIRS_90th',
 '02VIIRS_MAX',
 '02VIIRS_90th',
 '11VIIRS_75th',
 '01VIIRS_MAX',
 '12VIIRS_MAX']

In [ ]:
#data[data['electric_category'] == 0.0]['01VIIRS_MED'].hist(density = True, figsize=[6,6], bins=np.arange(0, 5, .1))

In [ ]:
# # remove negatives! 
# import numpy
# from matplotlib import pyplot

# x = data[data['electric_category'] == 1.0]['01VIIRS_MED'].dropna()
# y = data[data['electric_category'] == 0.0]['01VIIRS_MED'].dropna()

# bins=np.arange(0, 5, .1)
# pyplot.figure(figsize=(13,8))
# pyplot.hist(x, bins, alpha=0.5, label='Electrified', density = True)
# pyplot.hist(y, bins, alpha=0.5, label='Unelectrified', density = True)
# pyplot.legend(loc='upper right')
# pyplot.show()